<a href="https://colab.research.google.com/github/Tmol22/movie-recommendation/blob/database/Copy_of_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline

In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time
import asyncio
import ipywidgets as widgets

In [4]:
import pandas as pd

In [5]:
#Loading the dataset from github
url = "https://raw.githubusercontent.com/alooperalta/movieRecommendation/master/content/ml-latest-small/movies.csv"
movies = pd.read_csv(url)
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies.tail()

,movieId,title,genres
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
9741,193609,Andrew Dice Clay: Dice Rules (1991),Comedy


In [7]:
movies.describe()

,movieId
count,9742.000000
mean,42200.353623
std,52160.494854
min,1.000000
25%,3248.250000
50%,7300.000000
75%,76232.000000
max,193609.000000


In [8]:
movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [9]:
movies['genres']

0       Adventure|Animation|Children|Comedy|Fantasy
1                        Adventure|Children|Fantasy
2                                    Comedy|Romance
3                              Comedy|Drama|Romance
4                                            Comedy
                           ...                     
9737                Action|Animation|Comedy|Fantasy
9738                       Animation|Comedy|Fantasy
9739                                          Drama
9740                               Action|Animation
9741                                         Comedy
Name: genres, Length: 9742, dtype: object

In [10]:
movie_title = movies['title']
Genres = movies['genres'].str.get_dummies(sep='|')
Genres

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9738,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9739,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
9740,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
#Merge movies dataframe and genres dataframe together
movies = pd.concat([movies, Genres], axis=1)
movies.head()

,movieId,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
movies.drop(['genres','(no genres listed)'],axis=1,inplace=True)
movies.head()

,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Data Analysis

In [13]:
movies.describe()

,movieId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
count,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000
mean,42200.353623,0.187641,0.129645,0.062718,0.068158,0.385547,0.123075,0.045165,0.447649,0.079963,0.008930,0.100390,0.016218,0.034285,0.058817,0.163827,0.100595,0.194416,0.039212,0.017142
std,52160.494854,0.390445,0.335930,0.242468,0.252031,0.486749,0.328541,0.207677,0.497277,0.271250,0.094083,0.300535,0.126321,0.181968,0.235295,0.370137,0.300808,0.395771,0.194108,0.129808
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3248.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7300.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,76232.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,193609.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
movies.columns

Index(['movieId', 'title', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
       'War', 'Western'],
      dtype='object')

In [15]:
Action=movies.loc[movies['Action']==1]
print(f'Total no. of Action movies: {len(Action)}')
Action.head()

Total no. of Action movies: 1828


,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
5,6,Heat (1995),1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
8,9,Sudden Death (1995),1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,GoldenEye (1995),1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
14,15,Cutthroat Island (1995),1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
19,20,Money Train (1995),1,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,1,0,0


In [16]:
Adventure=movies.loc[movies['Adventure']==1]
print(f'Total no. of Adventure movies: {len(Adventure)}')
Adventure.head()

Total no. of Adventure movies: 1263


,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,8,Tom and Huck (1995),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,GoldenEye (1995),1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
12,13,Balto (1995),0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
Adventure= movies[movies['Adventure'] ==1]
print(f'Total no. of Adventure movies: {len(Adventure)}')
Adventure.head()

Total no. of Adventure movies: 1263


,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,8,Tom and Huck (1995),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,GoldenEye (1995),1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
12,13,Balto (1995),0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**MODEL CREATION**

In [18]:
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
import random

In [19]:
movies.head()

,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
X=movies.drop('title',axis=1)
n_clusters=170
recommender=KMeans(n_clusters=n_clusters,random_state=1)

In [21]:
labels=recommender.fit_predict(X)
labels

array([106, 106, 106, ...,  27,  27,  27], dtype=int32)

In [22]:
for i in labels:
  print(i)

Streaming output truncated to the last 5000 lines.
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146

In [23]:
dict(one=1, two=2)

{'one': 1, 'two': 2}

In [24]:
x = dict()
x[2] = 3
x

{2: 3}

In [25]:
for i in range(20):
  print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [26]:
rndi=dict()
for i in range(n_clusters):
    rndi[i]=[]
rndi

{0: [],
 1: [],
 2: [],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [],
 8: [],
 9: [],
 10: [],
 11: [],
 12: [],
 13: [],
 14: [],
 15: [],
 16: [],
 17: [],
 18: [],
 19: [],
 20: [],
 21: [],
 22: [],
 23: [],
 24: [],
 25: [],
 26: [],
 27: [],
 28: [],
 29: [],
 30: [],
 31: [],
 32: [],
 33: [],
 34: [],
 35: [],
 36: [],
 37: [],
 38: [],
 39: [],
 40: [],
 41: [],
 42: [],
 43: [],
 44: [],
 45: [],
 46: [],
 47: [],
 48: [],
 49: [],
 50: [],
 51: [],
 52: [],
 53: [],
 54: [],
 55: [],
 56: [],
 57: [],
 58: [],
 59: [],
 60: [],
 61: [],
 62: [],
 63: [],
 64: [],
 65: [],
 66: [],
 67: [],
 68: [],
 69: [],
 70: [],
 71: [],
 72: [],
 73: [],
 74: [],
 75: [],
 76: [],
 77: [],
 78: [],
 79: [],
 80: [],
 81: [],
 82: [],
 83: [],
 84: [],
 85: [],
 86: [],
 87: [],
 88: [],
 89: [],
 90: [],
 91: [],
 92: [],
 93: [],
 94: [],
 95: [],
 96: [],
 97: [],
 98: [],
 99: [],
 100: [],
 101: [],
 102: [],
 103: [],
 104: [],
 105: [],
 106: [],
 107: [],
 108: [],
 109: [],
 110: [],


In [27]:
len(labels)

9742

In [28]:
for i in range(len(labels)):
  print(labels[i])

Streaming output truncated to the last 5000 lines.
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
113
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146
146

In [29]:
for i in range(len(labels)):
    rndi[labels[i]].append(movie_title[i])
rndi

{0: ["Barney's Great Adventure (1998)",
  'Big One, The (1997)',
  'Chinese Box (1997)',
  'Lost in Space (1998)',
  'Mercury Rising (1998)',
  'Spanish Prisoner, The (1997)',
  'City of Angels (1998)',
  'Last Days of Disco, The (1998)',
  'Odd Couple II, The (1998)',
  'My Giant (1998)',
  'He Got Game (1998)',
  'Gingerbread Man, The (1998)',
  'Live Flesh (Carne trémula) (1997)',
  'Zero Effect (1998)',
  'Borrowers, The (1997)',
  "Krippendorf's Tribe (1998)",
  'Kurt & Courtney (1998)',
  'Real Blonde, The (1997)',
  'Mr. Nice Guy (Yat goh ho yan) (1997)',
  "Taste of Cherry (Ta'm e guilass) (1997)",
  'Character (Karakter) (1997)',
  'Species II (1998)',
  'Major League: Back to the Minors (1998)',
  'Big Hit, The (1998)',
  'Tarzan and the Lost City (1998)',
  'Dancer, Texas Pop. 81 (1998)',
  'Misérables, Les (1998)',
  'Clockwatchers (1997)',
  'Deep Impact (1998)',
  'Quest for Camelot (1998)',
  'Godzilla (1998)',
  'Bulworth (1998)',
  'Fear and Loathing in Las Vegas (1998

In [30]:
movie_title

0                                Toy Story (1995)
1                                  Jumanji (1995)
2                         Grumpier Old Men (1995)
3                        Waiting to Exhale (1995)
4              Father of the Bride Part II (1995)
                          ...                    
9737    Black Butler: Book of the Atlantic (2017)
9738                 No Game No Life: Zero (2017)
9739                                 Flint (2017)
9740          Bungo Stray Dogs: Dead Apple (2018)
9741          Andrew Dice Clay: Dice Rules (1991)
Name: title, Length: 9742, dtype: object

In [31]:
def getRecom(x):
    ind=movie_title[movie_title==x].index[0]
    choice=random.sample(rndi[labels[ind]],5)
    print('\nFinding Recommendations Similar to', x,'...')
    time.sleep(2)
    print('\nTop Matches for',x, 'are:\n' )
    for i in choice:
        if (i==x):
            print(random.sample(rndi[labels[ind]],1)[0]) 
        else:
            print(i)
    print("")

In [32]:
getRecom('Jumanji (1995)')


Finding Recommendations Similar to Jumanji (1995) ...

Top Matches for Jumanji (1995) are:

Boys on the Side (1995)
Poison Ivy II (1996)
Heat (1995)
Babe (1995)
Nobody Loves Me (Keiner liebt mich) (1994)



In [33]:
getRecom('Running Scared (1986)')


Finding Recommendations Similar to Running Scared (1986) ...

Top Matches for Running Scared (1986) are:

Holy Man (1998)
One True Thing (1998)
Allnighter, The (1987)
Peggy Sue Got Married (1986)
Mona Lisa (1986)



In [34]:
widgets.Button?

In [35]:
movie_title

0                                Toy Story (1995)
1                                  Jumanji (1995)
2                         Grumpier Old Men (1995)
3                        Waiting to Exhale (1995)
4              Father of the Bride Part II (1995)
                          ...                    
9737    Black Butler: Book of the Atlantic (2017)
9738                 No Game No Life: Zero (2017)
9739                                 Flint (2017)
9740          Bungo Stray Dogs: Dead Apple (2018)
9741          Andrew Dice Clay: Dice Rules (1991)
Name: title, Length: 9742, dtype: object

In [36]:
def model():
    o = movie_title
    opt = pd.concat([pd.Series(['Select an Option']),o])
    dropdown = widgets.Dropdown(
        options = opt,
        description='chose your favorite Movie:',
        disabled=False,
    )
    btn = widgets.Button(description='Find movies')
    display(dropdown)
    display(btn)
    def btn_eventhandler(obj):
      if(dropdown.value!='Select an Option'):
        getRecom(dropdown.value)
        btn.on_click(btn_eventhandler)

In [37]:
model()

Dropdown(description='chose your favorite Movie:', options=('Select an Option', 'Toy Story (1995)', 'Jumanji (…

Button(description='Find movies', style=ButtonStyle())

In [38]:
import pickle
#save the trained model with pickle
filename = 'recommender.pkl'
pickle_out = open(filename, 'wb')
pickle.dump(recommender , pickle_out)
pickle_out.close()

In [39]:
# Load the saved model
import pickle
filename = 'recommender.pkl'
pickle_in = open(filename, 'rb')
loaded_model = pickle.load(pickle_in)
loaded_model

KMeans(n_clusters=170, random_state=1)